In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests

from bs4 import BeautifulSoup


In [2]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(List_url)

In [3]:
soup = BeautifulSoup(response.text, "lxml")

# parsing the table part of the response by looking at wikitable sortable class-type
postal_table = soup.find(class_="wikitable sortable")

# building the initial dataframe from table's contents 
table_rows = postal_table.find_all('tr')
row_values = []
for tr in table_rows:
    td = tr.find_all('td')
    row_text = [tr.text.strip() for tr in td if tr.text.strip()]
    if row_text:
        row_values.append(row_text)

toronto_df = pd.DataFrame(row_values, columns=["PostalCode", "Borough", "Neighborhood"])
# toronto_df.head(10)
toronto_df.shape

(180, 3)

In [4]:
# ignoring cells with a Borough that is Not assigned.
borough_df = toronto_df[toronto_df.Borough != 'Not assigned']

# replacing 'Not assigned' neighborhood value with the corresponding Borough value
borough_df['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)


# combining neighborhoods with the same PostalCode into single row 
combined_df = borough_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index(drop=False)


print(combined_df.shape)
combined_df.head(10)

(103, 3)


c:\users\kumud.kharel\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
# geocoder seems to be very unreliable, so we can use the link to the csv file to get latitude and longitude
url = 'http://cocl.us/Geospatial_data'
lat_long_df = pd.read_csv(url)

# since the latitude-longitude table has the same data ordered as of combined_df, we can just create a new dataframe
# with the required columns
detailed_df = pd.DataFrame({'PostalCode':combined_df['PostalCode'], 
                            'Borough':combined_df['Borough'], 
                            'Neighborhood':combined_df['Neighborhood'], 
                            'Latitude':lat_long_df['Latitude'], 
                            'Longitude':lat_long_df['Longitude']})


print("Toronto has a total of {} boroughs and {} neighborhoods.".format(len(detailed_df.Borough.unique()), len(detailed_df.Neighborhood.unique())))

Toronto has a total of 10 boroughs and 99 neighborhoods.


In [6]:

# Analysing number of postal codes in '** toronto' borough
toronto_borough = ['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto']
for tor in toronto_borough:
    print("{} has a total of {} postal codes.".format(tor, detailed_df[detailed_df['Borough'] == tor].PostalCode.count()))

Downtown Toronto has a total of 19 postal codes.
Central Toronto has a total of 9 postal codes.
West Toronto has a total of 6 postal codes.
East Toronto has a total of 5 postal codes.


In [7]:
d_t = detailed_df[detailed_df['Borough'] == 'Downtown Toronto']
c_t = detailed_df[detailed_df['Borough'] == 'Central Toronto']
w_t = detailed_df[detailed_df['Borough'] == 'West Toronto']
e_t = detailed_df[detailed_df['Borough'] == 'East Toronto']

combined = pd.concat([d_t, c_t, w_t, e_t], sort=False)
toronto_dataframe = combined.reset_index(drop=True)

print(toronto_dataframe.shape)
toronto_dataframe.head(10)

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
